In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from itertools import product

In [2]:
data = load_iris()
X, y = data.data, data.target

In [3]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [5]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

In [6]:
class MLP:
    def __init__(self, input_size, hidden_sizes, output_size, learning_rate):
        self.learning_rate = learning_rate
        self.weights = []
        self.biases = []

        layer_sizes = [input_size] + hidden_sizes + [output_size]
        for i in range(len(layer_sizes) - 1):
            self.weights.append(np.random.randn(layer_sizes[i], layer_sizes[i+1]) * 0.1)
            self.biases.append(np.zeros((1, layer_sizes[i+1])))

    def forward(self, X):
        self.a = [X]
        for i in range(len(self.weights) - 1):
            z = np.dot(self.a[-1], self.weights[i]) + self.biases[i]
            self.a.append(sigmoid(z))
        z = np.dot(self.a[-1], self.weights[-1]) + self.biases[-1]
        self.a.append(z)
        return self.a[-1]

    def backward(self, y_true):
        deltas = [self.a[-1] - y_true.reshape(-1, 1)]
        for i in reversed(range(len(self.weights) - 1)):
            deltas.append(deltas[-1].dot(self.weights[i + 1].T) * sigmoid_derivative(self.a[i + 1]))
        deltas.reverse()

        for i in range(len(self.weights)):
            self.weights[i] -= self.learning_rate * self.a[i].T.dot(deltas[i])
            self.biases[i] -= self.learning_rate * np.mean(deltas[i], axis=0, keepdims=True)

    def train(self, X, y, epochs=1000):
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = np.mean((y_pred - y.reshape(-1, 1))**2)
            self.backward(y)

    def predict(self, X):
        y_pred = self.forward(X)
        return np.round(y_pred).flatten().astype(int)

In [7]:
def evaluate(model, X, y):
    y_pred = model.predict(X)
    accuracy = np.mean(y_pred == y)
    return accuracy

In [8]:
# Configuração do grid search
camadas_e_neuronios = [[5], [10], [20], [5, 5],[20,20], [10, 10]]
learning_rates = [0.1, 0.01, 0.001, 0.0001, 0.00001]
epochs = 100
best_accuracy = -1
best_params = {}

In [9]:
# Realizar grid search
for camadas, lr in product(camadas_e_neuronios, learning_rates):
    print(f"Treinando MLP com camadas ocultas: {camadas}, taxa de aprendizado: {lr}")
    mlp = MLP(input_size=4, hidden_sizes=camadas, output_size=1, learning_rate=lr)
    mlp.train(X_train, y_train, epochs=epochs)
    accuracy = evaluate(mlp, X_test, y_test)
    print(f"Acurácia: {accuracy*100:.2f}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = {'camadas e neuronios': camadas, 'taxa de aprendizado': lr}

Treinando MLP com camadas ocultas: [5], taxa de aprendizado: 0.1
Acurácia: 0.00
Treinando MLP com camadas ocultas: [5], taxa de aprendizado: 0.01
Acurácia: 97.78
Treinando MLP com camadas ocultas: [5], taxa de aprendizado: 0.001
Acurácia: 93.33
Treinando MLP com camadas ocultas: [5], taxa de aprendizado: 0.0001
Acurácia: 28.89
Treinando MLP com camadas ocultas: [5], taxa de aprendizado: 1e-05
Acurácia: 42.22
Treinando MLP com camadas ocultas: [10], taxa de aprendizado: 0.1


C:\Users\roney\AppData\Local\Temp\ipykernel_13428\3668515781.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
C:\Users\roney\AppData\Local\Temp\ipykernel_13428\3551417472.py:39: RuntimeWarning: invalid value encountered in cast
  return np.round(y_pred).flatten().astype(int)


Acurácia: 0.00
Treinando MLP com camadas ocultas: [10], taxa de aprendizado: 0.01
Acurácia: 0.00
Treinando MLP com camadas ocultas: [10], taxa de aprendizado: 0.001
Acurácia: 97.78
Treinando MLP com camadas ocultas: [10], taxa de aprendizado: 0.0001
Acurácia: 28.89
Treinando MLP com camadas ocultas: [10], taxa de aprendizado: 1e-05
Acurácia: 42.22
Treinando MLP com camadas ocultas: [20], taxa de aprendizado: 0.1
Acurácia: 0.00
Treinando MLP com camadas ocultas: [20], taxa de aprendizado: 0.01
Acurácia: 0.00
Treinando MLP com camadas ocultas: [20], taxa de aprendizado: 0.001
Acurácia: 93.33
Treinando MLP com camadas ocultas: [20], taxa de aprendizado: 0.0001
Acurácia: 28.89
Treinando MLP com camadas ocultas: [20], taxa de aprendizado: 1e-05
Acurácia: 42.22
Treinando MLP com camadas ocultas: [5, 5], taxa de aprendizado: 0.1


C:\Users\roney\AppData\Local\Temp\ipykernel_13428\3551417472.py:34: RuntimeWarning: overflow encountered in square
  loss = np.mean((y_pred - y.reshape(-1, 1))**2)
C:\Users\roney\AppData\Local\Temp\ipykernel_13428\3551417472.py:24: RuntimeWarning: invalid value encountered in multiply
  deltas.append(deltas[-1].dot(self.weights[i + 1].T) * sigmoid_derivative(self.a[i + 1]))


Acurácia: 28.89
Treinando MLP com camadas ocultas: [5, 5], taxa de aprendizado: 0.01
Acurácia: 95.56
Treinando MLP com camadas ocultas: [5, 5], taxa de aprendizado: 0.001
Acurácia: 28.89
Treinando MLP com camadas ocultas: [5, 5], taxa de aprendizado: 0.0001
Acurácia: 28.89
Treinando MLP com camadas ocultas: [5, 5], taxa de aprendizado: 1e-05
Acurácia: 42.22
Treinando MLP com camadas ocultas: [20, 20], taxa de aprendizado: 0.1
Acurácia: 28.89
Treinando MLP com camadas ocultas: [20, 20], taxa de aprendizado: 0.01
Acurácia: 71.11
Treinando MLP com camadas ocultas: [20, 20], taxa de aprendizado: 0.001
Acurácia: 28.89
Treinando MLP com camadas ocultas: [20, 20], taxa de aprendizado: 0.0001
Acurácia: 28.89
Treinando MLP com camadas ocultas: [20, 20], taxa de aprendizado: 1e-05
Acurácia: 42.22
Treinando MLP com camadas ocultas: [10, 10], taxa de aprendizado: 0.1
Acurácia: 28.89
Treinando MLP com camadas ocultas: [10, 10], taxa de aprendizado: 0.01
Acurácia: 95.56
Treinando MLP com camadas ocu

In [10]:
print(f"Melhores parâmetros: {best_params}, com acurácia de {best_accuracy*100:.2f}")

Melhores parâmetros: {'camadas e neuronios': [5], 'taxa de aprendizado': 0.01}, com acurácia de 97.78
